In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/MyDrive/Thesis/ASD

/content/gdrive/MyDrive/Thesis/ASD


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import sys
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
import torch.optim as optim
from sklearn.metrics import confusion_matrix
import functools
import numpy.ma as ma # for masked arrays
import glob
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tqdm.notebook import tqdm
from itertools import groupby
import sklearn

In [ ]:
from sklearn.model_selection import train_test_split
from captum.attr import IntegratedGradients
from captum.attr import Saliency
from captum.attr import DeepLift
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
from captum.attr import Saliency
import torchvision

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-4-1390d13c0cdd>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


### Helper functions for computing correlations

In [ ]:
# Auto Encoder and Classifier
class Network(nn.Module):
    def __init__(self, num_inputs = 19900):
        super(Network, self).__init__()

        self.num_inputs = num_inputs

        self.fc_encoder = nn.Sequential (
                nn.Linear(self.num_inputs,2048),
                nn.Tanh(),
                nn.Linear(2048,512),
                nn.Tanh())

        self.fc_decoder = nn.Sequential (
                nn.Linear(512,2048),
                nn.Tanh(),
                nn.Linear(2048,self.num_inputs),
                nn.Tanh())

        self.classifier = nn.Sequential (
            nn.Dropout(p=0.25),
            nn.Linear(512, 1),
        )

        self.sigmoid = nn.Sigmoid()

    def forward(self, x, eval_classifier = True):

        x = self.fc_encoder(x)
        if eval_classifier:
            x_logit = self.classifier(x)   #   .squeeze(1)
            # x_logit = self.sigmoid(x_logit)
            return x_logit

        x = self.fc_decoder(x)
        return x

In [ ]:
all_corr = pickle.load(open('./data/SFC_CC200.pkl', 'rb'))

# Using Intergrated Gradients Algorithm

In [ ]:
def attribute_image_features(algorithm, inputs):
    # model.zero_grad()
    # model.eval()
    tensor_attributions = algorithm.attribute(inputs = inputs, return_convergence_delta=True)
    return tensor_attributions

attributions = []
all_folds = pickle.load(open('./data/All_Folds_Subjects.pkl', 'rb'))

for fold in range(10):
    fold_weights = torch.load(f'./data/Weights/Fold_{fold+1}.pth', map_location=torch.device('cpu'))  # Loading the saved weigths
    best_clf_model = Network(num_inputs = 19900)
    best_clf_model.load_state_dict(fold_weights)
    best_clf_model = best_clf_model.to('cpu')

    test_subjects = all_folds[fold]['test']
    x_asd, y_asd = [], []
    for sample in test_subjects :
        if(all_corr[sample][1] == 1):         # Applying IG only on the correctly predicted autism samples.
          x_asd.append(all_corr[sample][0])
          y_asd.append(all_corr[sample][1])
    print('Number of ASD subjects in test set : ', len(x_asd))

    x_asd = torch.tensor(x_asd, dtype=torch.float)
    y_asd = np.array(y_asd)

    y_asd_pred = best_clf_model(x_asd)
    y_asd_pred = torch.sigmoid(y_asd_pred)
    y_asd_pred = y_asd_pred.detach().cpu().numpy()
    y_asd_pred = np.round(y_asd_pred)
    y_asd_pred = np.squeeze(y_asd_pred, axis = 1)

    right_indices = np.where(y_asd_pred == 1)    # Finding the indices of correctly predicted autism samples
    x_asd_ig = x_asd[right_indices]
    print('Number of correctly predicted ASD subjects : ', len(x_asd_ig))

    ig_asd = IntegratedGradients(best_clf_model)        # IG algorithm
    grads_asd, delta_asd = attribute_image_features(ig_asd, inputs = x_asd_ig)
    grads_asd = torch.mean(grads_asd, axis = 0)
    grads_asd = np.array(grads_asd)
    attributions.append(grads_asd)

attributions = np.array(attributions)
print("Attributions shape : ", attributions.shape)


Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  33
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  28
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  27
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  27
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  34
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  35
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  25
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  35
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  18
Number of ASD subjects in test set :  41
Number of correctly predicted ASD subjects :  18
Attributions shape :  (10, 19900)


In [ ]:
rois_count = {}    # {roi : number of times it is repeated in all 10 folds}

for grads_asd in attributions :

    attr_vals_asd = grads_asd.copy()
    thresh = np.percentile(attr_vals_asd, 99)
    attr_vals_asd = np.where(attr_vals_asd > thresh,  1 , 0)  # Thresholding
    corr_matrix_asd = np.zeros((200,200))
    corr_matrix_asd[np.triu_indices(200, 1)] = attr_vals_asd     # Reconstruction of Corr Matrix
    corr_matrix_asd[np.tril_indices(200, -1)] = attr_vals_asd
    print('Number of unique elements in corr_matrix : ', np.unique(corr_matrix_asd, return_counts = True))

    max_sum_rows = np.sum(corr_matrix_asd, axis = 1)      # Sum of rows to find out which ROIs correlations with other rois mostly repeated.
    top_indices = np.argsort(max_sum_rows)            # Max value indices
    top_values = max_sum_rows[top_indices]      # Max values

    top20_indices  = top_indices[-20 : ]
    top20_values = top_values[-20 : ]

    print('Most repeated ROIS in ASD (Not index values): ', top20_indices + 1)
    print('Number of times ROIS repeated in ASD : ', top20_values)

    for index, roi in enumerate(top_indices):
        if(roi in rois_count):
            rois_count[roi] += top_values[index]
        else :
          rois_count[roi] = top_values[index]

rois_count_sorted = dict(sorted(rois_count.items(), key = lambda x : x[1], reverse = True))
print(rois_count_sorted)

Number of unique elements in corr_matrix :  (array([0., 1.]), array([39602,   398]))
Most repeated ROIS in ASD (Not index values):  [131   5  81 170  21 117 138 108  55 106  30  59 123  64 142  42  89 105
 177 147]
Number of times ROIS repeated in ASD :  [5. 5. 5. 5. 5. 5. 6. 6. 6. 6. 7. 7. 7. 7. 7. 7. 7. 8. 9. 9.]
Number of unique elements in corr_matrix :  (array([0., 1.]), array([39602,   398]))
Most repeated ROIS in ASD (Not index values):  [ 30  81 147  66  64  73 138 188 106 108 131 195  55 197 123 155 142  59
 177 200]
Number of times ROIS repeated in ASD :  [5. 5. 5. 5. 5. 5. 6. 6. 6. 6. 6. 6. 6. 6. 6. 7. 7. 7. 9. 9.]
Number of unique elements in corr_matrix :  (array([0., 1.]), array([39602,   398]))
Most repeated ROIS in ASD (Not index values):  [ 64 108  17 128  46  49 147 131  63 107 200  59  42 142 146  55 138 106
 123 177]
Number of times ROIS repeated in ASD :  [ 5.  5.  5.  5.  5.  5.  6.  6.  6.  6.  6.  7.  7.  7.  7.  8.  8.  8.
  8. 12.]
Number of unique elements in

In [ ]:
cc200_labels = pd.read_csv('./data/CC200_ROI_labels.csv')
# cc200_labels.head(5)
print('ASD Associated Regions : ')
rois = [177, 59, 142, 106, 123, 200, 138, 55, 108]   # New Ones corrected
asd_rois = cc200_labels[cc200_labels['ROI number'].isin(rois)]
display(asd_rois)

ASD Associated Regions : 


,ROI number,volume,center of mass,Dosenbach,AAL,Eickhoff-Zilles,Talairach-Tournoux,Harvard-Oxford
54,55,247,(0.3;16.3;32.3),"[""None"": 0.83]","[""Cingulum_Mid_L"": 0.32][""Cingulum_Mid_R"": 0.22][""Cingulum_Ant_L"": 0.21][""Cingulum_Ant_R"": 0.20]","[""Left Anterior Cingulate Cortex"": 0.32][""Left Middle Cingulate Cortex"": 0.24][""Right Middle Cingulate Cortex"": 0.22][""Right Anterior Cingulate Cortex"": 0.20]","[""Left Cingulate Gyrus"": 0.44][""Right Cingulate Gyrus"": 0.29][""Right Anterior Cingulate"": 0.13][""Left Anterior Cingulate"": 0.13]","[""Right Cingulate Gyrus; anterior division"": 0.52][""Left Cingulate Gyrus; anterior division"": 0.27][""Left Paracingulate Gyrus"": 0.13]"
58,59,240,(36.7;17.2;3.6),"[""None"": 0.92]","[""Insula_R"": 0.61][""Putamen_R"": 0.20][""Frontal_Inf_Tri_R"": 0.11]","[""Right Insula Lobe"": 0.64][""Right Putamen"": 0.15]","[""Right Insula"": 0.46][""Right Inferior Frontal Gyrus"": 0.40][""Right Claustrum"": 0.14]","[""Right Insular Cortex"": 0.46][""Right Frontal Operculum Cortex"": 0.33]"
105,106,205,(28.6;34.6;42.0),"[""None"": 0.92]","[""Frontal_Mid_R"": 0.72][""Frontal_Sup_R"": 0.28]","[""Right Middle Frontal Gyrus"": 0.80][""Right Superior Frontal Gyrus"": 0.20]","[""Right Middle Frontal Gyrus"": 0.61][""Right Superior Frontal Gyrus"": 0.39]","[""Right Frontal Pole"": 0.47][""Right Middle Frontal Gyrus"": 0.42][""Right Superior Frontal Gyrus"": 0.11]"
107,108,266,(-6.9;-93.4;1.1),"[""None"": 0.93]","[""Calcarine_L"": 0.75]","[""Left Calcarine Gyrus"": 0.61][""Left Lingual Gyrus"": 0.20]","[""Left Lingual Gyrus"": 0.51][""Left Cuneus"": 0.41]","[""Left Occipital Pole"": 0.55][""Left Intracalcarine Cortex"": 0.20][""Left Lingual Gyrus"": 0.12]"
122,123,292,(0.4;-15.1;51.8),"[""None"": 0.99]","[""Supp_Motor_Area_L"": 0.32][""Supp_Motor_Area_R"": 0.22][""Cingulum_Mid_R"": 0.22][""Cingulum_Mid_L"": 0.18]","[""Left Middle Cingulate Cortex"": 0.27][""Right Middle Cingulate Cortex"": 0.26][""Left SMA"": 0.22][""Right SMA"": 0.17]","[""Right Medial Frontal Gyrus"": 0.36][""Left Medial Frontal Gyrus"": 0.35][""Right Paracentral Lobule"": 0.12]","[""Right Precentral Gyrus"": 0.27][""Left Precentral Gyrus"": 0.21][""Right Juxtapositional Lobule Cortex (formerly Supplementary Motor Cortex)"": 0.18][""Left Juxtapositional Lobule Cortex (formerly Supplementary Motor Cortex)"": 0.17]"
137,138,314,(22.5;-69.6;-11.3),"[""None"": 0.92]","[""Lingual_R"": 0.38][""Fusiform_R"": 0.34][""Cerebelum_6_R"": 0.28]","[""Right Cerebellum"": 0.52][""Right Fusiform Gyrus"": 0.25][""Right Lingual Gyrus"": 0.18]","[""Right Declive"": 0.57][""Right Lingual Gyrus"": 0.19][""Right Fusiform Gyrus"": 0.11]","[""Right Occipital Fusiform Gyrus"": 0.47][""Right Lingual Gyrus"": 0.35][""None"": 0.12]"
141,142,187,(17.8;-89.5;22.7),"[""None"": 1.00]","[""Occipital_Sup_R"": 0.47][""Cuneus_R"": 0.38]","[""Right Cuneus"": 0.40][""Right Superior Occipital Gyrus"": 0.37][""Right Middle Occipital Gyrus"": 0.12]","[""Right Cuneus"": 0.83][""Right Middle Occipital Gyrus"": 0.17]","[""Right Occipital Pole"": 0.66][""Right Lateral Occipital Cortex; superior division"": 0.24]"
176,177,277,(-14.3;-74.2;-10.1),"[""None"": 0.94]","[""Lingual_L"": 0.54][""Cerebelum_6_L"": 0.34]","[""Left Cerebellum"": 0.41][""Left Lingual Gyrus"": 0.39][""Left Cerebellum"": 0.11]","[""Left Declive"": 0.51][""Left Lingual Gyrus"": 0.31]","[""Left Lingual Gyrus"": 0.59][""Left Occipital Fusiform Gyrus"": 0.31]"
199,200,211,(-41.9;-31.5;15.2),"[""None"": 0.88]","[""Temporal_Sup_L"": 0.51][""Rolandic_Oper_L"": 0.23][""Heschl_L"": 0.11]","[""Left Superior Temporal Gyrus"": 0.62][""Left Rolandic Operculum"": 0.14][""Left Heschls Gyrus"": 0.14]","[""Left Superior Temporal Gyrus"": 0.46][""Left Insula"": 0.27][""Left Transverse Temporal Gyrus"": 0.20]","[""Left Parietal Operculum Cortex"": 0.35][""Left Planum Temporale"": 0.33][""Left Heschl's Gyrus (includes H1 and H2)"": 0.22]"


# Using DeepLIFT Algorithm

In [ ]:
def attribute_image_features(algorithm, inputs):
    # model.zero_grad()
    # model.eval()
    tensor_attributions = algorithm.attribute(inputs = inputs, return_convergence_delta=True)
    return tensor_attributions

dl_attributions = []
all_folds = pickle.load(open('./data/All_Folds_Subjects.pkl', 'rb'))
for fold in range(10):
    fold_weights = torch.load(f'data/Weights/Fold_{fold+1}.pth', map_location=torch.device('cpu'))
    best_clf_model = Network(num_inputs = 19900)
    best_clf_model.load_state_dict(fold_weights)
    best_clf_model = best_clf_model.to('cpu')

    test_subjects = all_folds[fold]['test']
    x_asd, y_asd = [], []
    for sample in test_subjects :
        if(all_corr[sample][1] == 1):
          x_asd.append(all_corr[sample][0])
          y_asd.append(all_corr[sample][1])
    print('Number of ASD subjects in test set : ', len(x_asd))

    x_asd = torch.tensor(x_asd, dtype=torch.float)
    y_asd = np.array(y_asd)

    y_asd_pred = best_clf_model(x_asd)
    y_asd_pred = torch.sigmoid(y_asd_pred)
    y_asd_pred = y_asd_pred.detach().cpu().numpy()
    y_asd_pred = np.round(y_asd_pred)
    y_asd_pred = np.squeeze(y_asd_pred, axis = 1)

    right_indices = np.where(y_asd_pred == 1)
    x_asd_dl = x_asd[right_indices]
    print('Number of correctly predicted ASD subjects : ', len(x_asd_dl))

    dl_asd = DeepLift(best_clf_model)
    grads_asd, delta_asd = attribute_image_features(dl_asd, inputs = x_asd_dl)
    grads_asd = torch.mean(grads_asd, axis = 0)
    grads_asd = grads_asd.detach().cpu().numpy()
    dl_attributions.append(grads_asd)

dl_attributions = np.array(dl_attributions)
print("Attributions shape : ", dl_attributions.shape)


Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  33


/home/krishna/anaconda3/lib/python3.8/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/home/krishna/anaconda3/lib/python3.8/site-packages/captum/attr/_core/deep_lift.py:336: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(


Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  28
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  27
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  27
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  34
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  35
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  25
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  35
Number of ASD subjects in test set :  42
Number of correctly predicted ASD subjects :  18
Number of ASD subjects in test set :  41
Number of correctly predicted ASD subjects :  18
Attributions shape :  (10, 19900)


In [ ]:
rois_count = {}    # {roi : number of times it is repeated in all 10 folds}

for grads_asd in dl_attributions :

    attr_vals_asd = grads_asd.copy()
    thresh = np.percentile(attr_vals_asd, 99)
    attr_vals_asd = np.where(attr_vals_asd > thresh,  1 , 0)  # Thresholding
    corr_matrix_asd = np.zeros((200,200))
    corr_matrix_asd[np.triu_indices(200, 1)] = attr_vals_asd     # Reconstruction of Corr Matrix
    corr_matrix_asd[np.tril_indices(200, -1)] = attr_vals_asd
    print('Number of unique elements in corr_matrix : ', np.unique(corr_matrix_asd, return_counts = True))

    max_sum_rows = np.sum(corr_matrix_asd, axis = 1)      # Sum of rows to find out which ROIs correlations with other rois mostly repeated.
    top_indices = np.argsort(max_sum_rows)            # Max value indices
    top_values = max_sum_rows[top_indices]      # Max values

    top20_indices  = top_indices[-20 : ]
    top20_values = top_values[-20 : ]

    print('Most repeated ROIS in ASD (Not index values): ', top20_indices + 1)
    print('Number of times ROIS repeated in ASD : ', top20_values)

    for index, roi in enumerate(top_indices):
        if(roi in rois_count):
            rois_count[roi] += top_values[index]
        else :
          rois_count[roi] = top_values[index]

rois_count_sorted = dict(sorted(rois_count.items(), key = lambda x : x[1], reverse = True))
print(rois_count_sorted)

Number of unique elements in corr_matrix :  (array([0., 1.]), array([39602,   398]))
Most repeated ROIS in ASD (Not index values):  [179 106  21  81 167 170 117  30 138 108  55  42  59 123  64 142  89 105
 177 147]
Number of times ROIS repeated in ASD :  [5. 5. 5. 5. 5. 5. 5. 6. 6. 6. 6. 6. 7. 7. 7. 7. 7. 8. 9. 9.]
Number of unique elements in corr_matrix :  (array([0., 1.]), array([39602,   398]))
Most repeated ROIS in ASD (Not index values):  [ 30  81 147  66  64  73 138 188 106 108 131 195  55 197 123 155 142  59
 177 200]
Number of times ROIS repeated in ASD :  [5. 5. 5. 5. 5. 5. 6. 6. 6. 6. 6. 6. 6. 6. 6. 7. 7. 7. 9. 9.]
Number of unique elements in corr_matrix :  (array([0., 1.]), array([39602,   398]))
Most repeated ROIS in ASD (Not index values):  [ 64 108  17 128  46  49 147 131  63 107 200  59  42 142 146  55 138 106
 123 177]
Number of times ROIS repeated in ASD :  [ 5.  5.  5.  5.  5.  5.  6.  6.  6.  6.  6.  7.  7.  7.  7.  8.  8.  8.
  8. 12.]
Number of unique elements in

In [ ]:
cc200_labels = pd.read_csv('./data/CC200_ROI_labels.csv')
# cc200_labels.head(5)
print('ASD Associated Regions : ')
rois = [177, 59, 142, 106, 123, 200, 138, 55, 108]
asd_rois = cc200_labels[cc200_labels['ROI number'].isin(rois)]
display(asd_rois)

ASD Associated Regions : 


,ROI number,volume,center of mass,Dosenbach,AAL,Eickhoff-Zilles,Talairach-Tournoux,Harvard-Oxford
54,55,247,(0.3;16.3;32.3),"[""None"": 0.83]","[""Cingulum_Mid_L"": 0.32][""Cingulum_Mid_R"": 0.22][""Cingulum_Ant_L"": 0.21][""Cingulum_Ant_R"": 0.20]","[""Left Anterior Cingulate Cortex"": 0.32][""Left Middle Cingulate Cortex"": 0.24][""Right Middle Cingulate Cortex"": 0.22][""Right Anterior Cingulate Cortex"": 0.20]","[""Left Cingulate Gyrus"": 0.44][""Right Cingulate Gyrus"": 0.29][""Right Anterior Cingulate"": 0.13][""Left Anterior Cingulate"": 0.13]","[""Right Cingulate Gyrus; anterior division"": 0.52][""Left Cingulate Gyrus; anterior division"": 0.27][""Left Paracingulate Gyrus"": 0.13]"
58,59,240,(36.7;17.2;3.6),"[""None"": 0.92]","[""Insula_R"": 0.61][""Putamen_R"": 0.20][""Frontal_Inf_Tri_R"": 0.11]","[""Right Insula Lobe"": 0.64][""Right Putamen"": 0.15]","[""Right Insula"": 0.46][""Right Inferior Frontal Gyrus"": 0.40][""Right Claustrum"": 0.14]","[""Right Insular Cortex"": 0.46][""Right Frontal Operculum Cortex"": 0.33]"
105,106,205,(28.6;34.6;42.0),"[""None"": 0.92]","[""Frontal_Mid_R"": 0.72][""Frontal_Sup_R"": 0.28]","[""Right Middle Frontal Gyrus"": 0.80][""Right Superior Frontal Gyrus"": 0.20]","[""Right Middle Frontal Gyrus"": 0.61][""Right Superior Frontal Gyrus"": 0.39]","[""Right Frontal Pole"": 0.47][""Right Middle Frontal Gyrus"": 0.42][""Right Superior Frontal Gyrus"": 0.11]"
107,108,266,(-6.9;-93.4;1.1),"[""None"": 0.93]","[""Calcarine_L"": 0.75]","[""Left Calcarine Gyrus"": 0.61][""Left Lingual Gyrus"": 0.20]","[""Left Lingual Gyrus"": 0.51][""Left Cuneus"": 0.41]","[""Left Occipital Pole"": 0.55][""Left Intracalcarine Cortex"": 0.20][""Left Lingual Gyrus"": 0.12]"
122,123,292,(0.4;-15.1;51.8),"[""None"": 0.99]","[""Supp_Motor_Area_L"": 0.32][""Supp_Motor_Area_R"": 0.22][""Cingulum_Mid_R"": 0.22][""Cingulum_Mid_L"": 0.18]","[""Left Middle Cingulate Cortex"": 0.27][""Right Middle Cingulate Cortex"": 0.26][""Left SMA"": 0.22][""Right SMA"": 0.17]","[""Right Medial Frontal Gyrus"": 0.36][""Left Medial Frontal Gyrus"": 0.35][""Right Paracentral Lobule"": 0.12]","[""Right Precentral Gyrus"": 0.27][""Left Precentral Gyrus"": 0.21][""Right Juxtapositional Lobule Cortex (formerly Supplementary Motor Cortex)"": 0.18][""Left Juxtapositional Lobule Cortex (formerly Supplementary Motor Cortex)"": 0.17]"
137,138,314,(22.5;-69.6;-11.3),"[""None"": 0.92]","[""Lingual_R"": 0.38][""Fusiform_R"": 0.34][""Cerebelum_6_R"": 0.28]","[""Right Cerebellum"": 0.52][""Right Fusiform Gyrus"": 0.25][""Right Lingual Gyrus"": 0.18]","[""Right Declive"": 0.57][""Right Lingual Gyrus"": 0.19][""Right Fusiform Gyrus"": 0.11]","[""Right Occipital Fusiform Gyrus"": 0.47][""Right Lingual Gyrus"": 0.35][""None"": 0.12]"
141,142,187,(17.8;-89.5;22.7),"[""None"": 1.00]","[""Occipital_Sup_R"": 0.47][""Cuneus_R"": 0.38]","[""Right Cuneus"": 0.40][""Right Superior Occipital Gyrus"": 0.37][""Right Middle Occipital Gyrus"": 0.12]","[""Right Cuneus"": 0.83][""Right Middle Occipital Gyrus"": 0.17]","[""Right Occipital Pole"": 0.66][""Right Lateral Occipital Cortex; superior division"": 0.24]"
176,177,277,(-14.3;-74.2;-10.1),"[""None"": 0.94]","[""Lingual_L"": 0.54][""Cerebelum_6_L"": 0.34]","[""Left Cerebellum"": 0.41][""Left Lingual Gyrus"": 0.39][""Left Cerebellum"": 0.11]","[""Left Declive"": 0.51][""Left Lingual Gyrus"": 0.31]","[""Left Lingual Gyrus"": 0.59][""Left Occipital Fusiform Gyrus"": 0.31]"
199,200,211,(-41.9;-31.5;15.2),"[""None"": 0.88]","[""Temporal_Sup_L"": 0.51][""Rolandic_Oper_L"": 0.23][""Heschl_L"": 0.11]","[""Left Superior Temporal Gyrus"": 0.62][""Left Rolandic Operculum"": 0.14][""Left Heschls Gyrus"": 0.14]","[""Left Superior Temporal Gyrus"": 0.46][""Left Insula"": 0.27][""Left Transverse Temporal Gyrus"": 0.20]","[""Left Parietal Operculum Cortex"": 0.35][""Left Planum Temporale"": 0.33][""Left Heschl's Gyrus (includes H1 and H2)"": 0.22]"
